<header style="padding:10px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>RETAIL: Broken Digital Journeys Demo</b>
<hr>
<br>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>    



<p style = 'font-size:16px;font-family:Arial'>This document provides a base demo script for showcasing the capabilities of CX 1-2-3 Broken Digital Journeys through a Jupyter Notebook using teradataml (Teradata Python package) and Plotly.  The target audience for either on-premise customers and/or Data Scientists.</p>


<p style = 'font-size:16px;font-family:Arial'>The nPath function scans a set of rows, looking for patterns that you specify. For each set of input rows that matches the pattern, nPath produces a single output row. The function provides a flexible pattern-matching capability that lets you specify complex patterns in the input data and define the values that are output for each matched input set.</p>
<p style = 'font-size:16px;font-family:Arial'>nPath is useful when your goal is to identify the paths that lead to an outcome. For example, you can use nPath to analyze:</p>

<li style = 'font-size:16px;font-family:Arial'>Web site click data, to identify paths that lead to sales over a specified amount</li>
<li style = 'font-size:16px;font-family:Arial'>Sensor data from industrial processes, to identify paths to poor product quality</li>
<li style = 'font-size:16px;font-family:Arial'>Healthcare records of individual patients, to identify paths that indicate that patients are at risk of developing conditions such as heart disease or diabetes</li>
<li style = 'font-size:16px;font-family:Arial'>Financial data for individuals, to identify paths that provide information about credit or fraud risks</li>
<p style = 'font-size:16px;font-family:Arial'>The output from the nPath function can be input to other ML Engine functions or to a visualization tool.</p>
<p style = 'font-size:16px;font-family:Arial'>This demo shows how to use Vantage nPath to analyze customer digital behavior, experience, and diagnosis of problems with customer experience and flow. It uses a simulated digital retail data set. Using this data we are conducting path analysis. Specifically, it has examples of looking at the most frequent customer journeys and identifying any common events that could potentially be stopping the customer from making a purchase.</p>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>1. Start by importing the required libraries and connecting to the Vantage system.</b></p>


<p style = 'font-size:18px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).</p>

In [ ]:
%%capture
# RESTART KERNEL AFTER THIS. NO NEED TO EXECUTE THIS AFTER RESTART.
!pip install teradataml --upgrade
!pip install colorlover

<p style = 'font-size:18px;font-family:Arial'><b>** Restart Kernel before moving to next cell </b></p>
<p style = 'font-size:18px;font-family:Arial'>Import the libraries mentioned in the below cell.</p>

In [ ]:
import json
import getpass
import os
import warnings
import datetime
from collections import defaultdict

import pandas as pd
import numpy as np

import teradataml.dataframe.dataframe as tdf
from teradataml.dataframe.dataframe import in_schema
from teradataml.context.context import create_context, remove_context, get_context
from teradataml.dataframe.copy_to import copy_to_sql
from teradataml.dataframe.fastload import fastload

from teradataml.analytics.sqle.Sessionize import Sessionize
from teradataml.analytics.sqle.NPath import NPath
from teradataml.dataframe.dataframe import DataFrame

from teradatasqlalchemy.types import *

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import defaultdict
import plotly.offline as offline
import colorlover as cl

offline.init_notebook_mode()

warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell. Begin running steps with Shift + Enter keys.</p> 

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
eng.execute('''SET query_band='DEMO=RetailBrokenDigitalJourneysPython.ipynb;' UPDATE FOR SESSION; ''')

<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>2. Getting Data for This Demo
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage.  You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage.  There are two statements in the following cell, and one is commented out.  You may switch which mode you choose by changing the comment string.</p>    


In [ ]:
#%run -i ../run_procedure.py "call get_data('DEMO_DigitalEvents_cloud');"
# Takes about 12 seconds
%run -i ../run_procedure.py "call get_data('DEMO_DigitalEvents_local');"
# Takes about 20 seconds

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>5. Analyze the raw data set</b></p>

<p style = 'font-size:16px;font-family:Arial'>Create a DataFrame to get the data from the tablecreated.</p>




In [ ]:
df = tdf.DataFrame(in_schema('DEMO_DigitalEvents', 'Digital_Retail_Events'))
df.head(5)

In [ ]:
from teradataml import ConvertTo
converted_data = ConvertTo(data = df,
                           target_columns = ['Customer_Id','datestamp', 'event','session_id'],
                           target_datatype = ["integer","timestamp","VARCHAR(charlen=20,charset=LATIN,casespecific=NO)","integer"])
converted_data.result

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6. Identifying Broken Digital Journy </b></p>

<p style = 'font-size:18px;font-family:Arial'>This section will walkthrough conducting path analysis using retail data. Specifically, it has examples of looking at the most frequent customer journeys and identifying any common events that could potentially be stopping the customer from making a purchase.</p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.1. Sankey Charts</b></p>
<p style = 'font-size:16px;font-family:Arial'>Sankey charts can help visualize pathways and volume of events on the  most common paths.  Teradata VantageCloud does have built-in visualizations, but if users don't have access to these capabilities, or perhaps there is a preference to work in Python, the Plotly package provides Sankey charting capabilities.  Below the Python function reads output from nPath as pandas dataframe and plots Sankey chart. This method will be used throughout for plotting Sankey.</p>

In [ ]:
#Convert Teradata nPath output to plotly Sankey
#can handle paths up to 999 links in length
import plotly.graph_objects as go
def sankeyPlot(res, title_text="Basic Sankey Path"):
    npath_pandas = res
    
    dataDict = defaultdict(int)
    eventDict = defaultdict(int)
    maxPath = npath_pandas['count_customer_id'].max()
    
    for index, row in npath_pandas.iterrows():
        rowList = row['path'].replace('[','').replace(']','').split(',')
        pathCnt = row['count_customer_id']
        pathLen = len(rowList)
        for i in range(len(rowList)-1):
            leftValue = str(100 + i + maxPath - pathLen) + rowList[i].strip()
            rightValue = str(100 + i + 1 + maxPath - pathLen) + rowList[i+1].strip()
            valuePair = leftValue + '+' + rightValue
            dataDict[valuePair] += pathCnt
            eventDict[leftValue] += 1
            eventDict[rightValue] += 1
    
    eventList = []
    for key,val in eventDict.items():
        eventList.append(key)
    
    sortedEventList = sorted(eventList)
    sankeyLabel = []
    for event in sortedEventList:
        sankeyLabel.append(event[3:])
    
    sankeySource = []
    sankeyTarget = []
    sankeyValue = []

    for key,val in dataDict.items():
        sankeySource.append(sortedEventList.index(key.split('+')[0]))
        sankeyTarget.append(sortedEventList.index(key.split('+')[1]))
        sankeyValue.append(val)
    
    sankeyColor = []
    for i in sankeyLabel:
        sankeyColor.append('blue')
    
    sankeyChart = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = 'black',
            width = 0.5
          ),
          label = sankeyLabel,
          color = sankeyColor
        ),
        link = dict(
            source = sankeySource,
            target = sankeyTarget,
            value = sankeyValue
        )
      )
    layout =  dict(
        title = title_text,
        font = dict(
          size = 10
        )
    )
    
    link = dict(source = sankeySource, target = sankeyTarget, value = sankeyValue, color='white')
    node = dict(label=sankeyLabel, pad=15, thickness=20, color='orange')
    data=go.Sankey(link=link, node=node)
    # plot 
    fig=go.Figure(data)
    fig.update_layout(
            hovermode ='closest',
            title = title_text,
            title_font_size=20,
            font = dict(size = 10, color = 'white'),
            plot_bgcolor='black',
            paper_bgcolor="#585958"
            )
    fig.show()  

    
    # fig = dict(data=[sankeyChart], layout=layout)
    # iplot(fig, validate=False)

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6.2 Calling a basic nPath function.</b></p>
<li style = 'font-size:16px;font-family:Arial'>The Vantage nPath analytic function scans a set of rows, looking for patterns.</li>
<li style = 'font-size:16px;font-family:Arial'>For each set of input rows that matches the pattern, nPath produces a single output row.</li>

<p style = 'font-size:16px;font-family:Arial'>This function allows for matching of complex patterns in the input data, as well as defining the output values for each matched set of rows.</p>

<p style = 'font-size:16px;font-family:Arial'>For the below example:
<p style = 'font-size:16px;font-family:Arial'>1. Pass the input data by reference.</p>
<p style = 'font-size:16px;font-family:Arial'>2. Provide partitioning (customer_id, session_id) and ordering columns.
<p style = 'font-size:16px;font-family:Arial'>3. Mode.<b>**OVERLAPPING**</b> vs.<b> **NONOVERLAPPING**</b>
    <li><b> **OVERLAPPING**</b> finds every occurrence of the match, regardless of the current row being part of a previous match.</li>
    <li><b> **NONOVERLAPPING** </b>starts matching again at the row that follows the previous match.</li>
<p style = 'font-size:16px;font-family:Arial'>4. Symbols.  Create a set of column expression aliases that can be assembled into a pattern to match.
 <li>Example: "EVENT = 'Home Page' as P" will alias a match on the EVENT column when the content equals 'Home Page'.</li></p>
<p style = 'font-size:16px;font-family:Arial'>5. Pattern.  Compose a pattern to search for across the rows of events.  This pattern is composed of Symbols and directives.
 <li>Example: '^P' uses a directive ^ to indicate the P Symbol must occur at the beginning of the group of rows.</li></p>
<p style = 'font-size:16px;font-family:Arial'>6. Result.  Since nPath emits a single row per group-of-row matches, Result indicates what columns make up this row and how to aggregate the data.</p>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6.3 Path to Sales Conversion</b></p>
<p style = 'font-size:16px;font-family:Arial'>Let's start the analyis by taking a look at what a successful path looks like.  This is accomplished by running the Vantage nPath analytic function and specifying the pattern as from Any Event to Sales Conversion.</p>

In [ ]:
copy_to_sql(converted_data.result, table_name = 'npath_data', schema_name = 'demo_user', 
            if_exists = 'replace')

In [ ]:
#Create two symbols and assemble them with directives:
# 1. EVENT Column match the string 'Sales Conversion' as B and all other EVENT as O
# Pattern directs a range of any row (O) between 1 and 4 times preceding 'Sales Conversion' (B) - O{1,4}.B

npath_sessions = NPath(data1 = DataFrame('"demo_user"."npath_data"'), 
                      data1_partition_column = ['Customer_Id','session_id'], 
                      data1_order_column = 'datestamp', 
                      mode = 'NONOVERLAPPING', 
                      symbols = ['event in (\'Sales Conversion\') as B', 'event not in (\'Sales Conversion\') as O'], 
                      pattern = 'O{1,4}.B', 
                      result = ['FIRST (customer_id OF O) AS customer_id',
                                'ACCUMULATE (event of any(O,B) ) AS path'])

npath_sessions.result\
                    .groupby(['path'])\
                    .count()\
                    .sort('count_customer_id',ascending=False)\
                    .to_pandas()\
                    .head(10)

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Path to Sales Conversion Visualization</b></p>
<p style = 'font-size:16px;font-family:Arial'>In the table view you can see that the most common last step before Sales Conversion is Add to Cart.  Visualizing the paths on a Sankey diagram is an effective way to see common digital customer journey patterns.</p>

In [ ]:
#warnings.simplefilter(action='ignore', category=DeprecationWarning)
res = npath_sessions.result\
                    .groupby(['path'])\
                    .count()\
                    .sort('count_customer_id',ascending=False)\
                    .head(50)\
                    .to_pandas()
sankeyPlot(res,"Path to Sales Conversion")

<p style = 'font-size:16px;font-family:Arial'>The TOP 3 events that led to Sales Conversion are <b>Add To Cart</b> ,<b>Product Search</b> and <b>Availability Status.</b></p>

<p style = 'font-size:16px;font-family:Arial'>To check the details of any<b> path</b> or <b> node </b>we can move the mouse pointer over it and check details. For example if you mouse the pointer over the parth wihich is having the largest width going towards the right most node(Sales Conversion) it shows <b> 55.0, source:Add To Cart, target: Sales Conversion</b>. It means there were 55 events where after adding the product Cart the next step was that the product was bought, which led to Sales.</p>
<p style = 'font-size:16px;font-family:Arial'>When the pointer is moved over a Node, for example when the pointer is on the last Node at the right <b>Availability Status</b> it shows <b>incoming flow count: 4</b> and <b>outgoing flow count: 1</b> which means that there are 4 different events which lead to <b>Availability Status</b> after which the next event is <b>Sales Conversion</b>.

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6.4 Path to Cart Abandoned</b></p>
<p style = 'font-size:16px;font-family:Arial'>The data available tracks customers who abandon their carts, change the Ending Pattern and see if you can learn anything from the common paths to the Cart Abandoned event.</p>

In [ ]:
#Create two symbols and assemble them with directives:
# 1. EVENT Column match the string 'Cart Abandoned' as B and all other EVENT as O
# Pattern directs a range of any row (O) between 1 and 4 times preceding 'Cart Abandoned' (B) - O{1,4}.B

npath_sessions = NPath(data1 = DataFrame('"demo_user"."npath_data"'), 
                      data1_partition_column = ['Customer_Id','session_id'], 
                      data1_order_column = 'datestamp', 
                      mode = 'NONOVERLAPPING', 
                      symbols = ['event in (\'Cart Abandoned\') as B', 'event not in (\'Cart Abandoned\') as O'], 
                      pattern = 'O{1,4}.B', 
                      result = ['FIRST (customer_id OF O) AS customer_id',
                                'ACCUMULATE (event of any(O,B) ) AS path'])

npath_sessions.result\
                    .groupby(['path'])\
                    .count()\
                    .sort('count_customer_id',ascending=False)\
                    .to_pandas()\
                    .head(10)

In [ ]:
res = npath_sessions.result\
                    .groupby(['path'])\
                    .count()\
                    .sort('count_customer_id',ascending=False)\
                    .head(50)\
                    .to_pandas()
sankeyPlot(res,"Path to Cart Abandoned")

<p style = 'font-size:16px;font-family:Arial'>The Cart was abondened by most of the customers of the <b>Search Not Found</b> event occurred.</p>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6.5 Path to Search Not Found</b></p>
<p style = 'font-size:16px;font-family:Arial'>As you can see from the common paths to Cart Abandoned - Search Not Found is experienced by many customers.  Explore this further by changing the ending pattern to Search Not Found.</p>

In [ ]:
#Create two symbols and assemble them with directives:
# 1. EVENT Column match the string 'Search Not Found' as B and all other EVENT as O
# Pattern directs a range of any row (O) between 1 and 4 times preceding 'Search Not Found' (B) - O{1,4}.B

npath_sessions = NPath(data1 = DataFrame('"demo_user"."npath_data"'), 
                      data1_partition_column = ['Customer_Id','session_id'], 
                      data1_order_column = 'datestamp', 
                      mode = 'NONOVERLAPPING', 
                      symbols = ['event in (\'Search Not Found\') as B', 'event not in (\'Search Not Found\') as O'], 
                      pattern = 'O{1,4}.B', 
                      result = ['FIRST (customer_id OF O) AS customer_id',
                                'ACCUMULATE (event of any(O,B) ) AS path'])

npath_sessions.result\
                    .groupby(['path'])\
                    .count()\
                    .sort('count_customer_id',ascending=False)\
                    .to_pandas()\
                    .head(10)

In [ ]:
res = npath_sessions.result\
                    .groupby(['path'])\
                    .count()\
                    .sort('count_customer_id',ascending=False)\
                    .head(50)\
                    .to_pandas()

sankeyPlot(res,"Path to Search Not Found")

<p style = 'font-size:16px;font-family:Arial'>The <b>Search Not Found</b> event occurred mainly after <b>Product Search</b> from the Home Page.</p>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6.6 Path From Search Not Found</b></p>
<p style = 'font-size:16px;font-family:Arial'>With a significant number of paths leading to Search Not found, as well as the number of customers on these paths, we need to explore more on this.</p>
<p style = 'font-size:16px;font-family:Arial'>
Now change to swap have 'Search Not Found' as the starting event and see where customers go to after this key event</p>

In [ ]:
#Create two symbols and assemble them with directives:
# 1. EVENT Column match the string 'Search Not Found' as A and all other EVENT as O
# Pattern directs a range of any row (O) between 1 and 4 times succeeding 'Search Not Found' (A) - A.O{1,4}

npath_sessions = NPath(data1 = DataFrame('"demo_user"."npath_data"'), 
                      data1_partition_column = ['Customer_Id','session_id'], 
                      data1_order_column = 'datestamp', 
                      mode = 'NONOVERLAPPING', 
                      symbols = ['event in (\'Search Not Found\') as A', 'true as O'], 
                      pattern = 'A.O{1,4}', 
                      result = ['FIRST (customer_id OF O) AS customer_id',
                                'ACCUMULATE (event of any(A,O) ) AS path'])

npath_sessions.result\
                    .groupby(['path'])\
                    .count()\
                    .sort('count_customer_id',ascending=False)\
                    .to_pandas()\
                    .head(10)

In [ ]:
res = npath_sessions.result\
                    .groupby(['path'])\
                    .count()\
                    .sort('count_customer_id',ascending=False)\
                    .head(50)\
                    .to_pandas()

sankeyPlot(res,"Path From Search Not Found")

<p style = 'font-size:16px;font-family:Arial'>The events that were followed by <b>Search Not Found</b> were mainly <b>Cart Abondoned</b> and <b>Availability Status</b>.</p>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6.7 Storing Customers on a Path</b></p>
<p style = 'font-size:16px;font-family:Arial'>With the above analysis, it can now be confirmed that there is definitely an issue with unsuccessful searches. From here business actions could be taken such as:</p>

<li style = 'font-size:16px;font-family:Arial'>Raising a defect for our Website Development team to focus on and with the data supporting this, it should be a high priority!</li>
<li style = 'font-size:16px;font-family:Arial'>Save a list of the customers on the dominant path so that additional analysis could be preformed and/or so that the Customer Care team can proactively reach out to the customers.</li>

<p style = 'font-size:16px;font-family:Arial'>The next section will demonstrate how to save a list of customers on any path of interest.</p>

In [ ]:
#Create symbols for each event in our desired path and assemble them with directives:
# 1. EVENT Column match the string 'Cart Abandoned' as A
# 2. EVENT Column match the string 'Search Not Found' as B
# 3. EVENT Column match the string 'Product Search' as C
# 4. EVENT Column match the string 'Home Page' as D
# 5. EVENT Column match the string 'Search Not Found' as E
# Pattern directs events in a particular order which in this case is E.D.C.B.A

npath_sessions = NPath(data1 = DataFrame('"demo_user"."npath_data"'), 
                      data1_partition_column = ['Customer_Id','session_id'], 
                      data1_order_column = 'datestamp', 
                      mode = 'NONOVERLAPPING', 
                      symbols = ['event = \'Search Not Found\' as E', 
                                 'event = \'Home Page\' as D', 
                                 'event = \'Product Search\' as C', 
                                 'event = \'Search Not Found\' as B', 
                                 'event = \'Cart Abandoned\' as A'], 
                      pattern = 'E.D.C.B.A', 
                      result = ['FIRST (customer_id of ANY (E,D,C,B,A)) AS customer_id',
                                'ACCUMULATE (event of ANY (E,D,C,B,A)) AS path'])

npath_sessions.result.head()

In [ ]:
# Now we will take the list of distinct customers who have taken this path and 
# store it in Vantage table for detailed analysis

res = npath_sessions.result\
              .groupby(['customer_id','path'])\
              .count()\
              .sort('count_path',ascending=False)\
              .to_pandas()\
              .reset_index()

# Writing clean_input dataframe into MEDIQAN_TEMP_INPUT_TABLE table in Vantage
copy_to_sql(df = res[['customer_id','path']], 
            table_name = 'CustomersOnPath', 
            #schema_name = 'target_db', # Uncomment & specify if not using the Vantage Live Default Database, which usually is your QLID.
            if_exists="replace")


In [ ]:
result_df = tdf.DataFrame('CustomersOnPath')
print("Number of records in CustomersOnPath : "+str(result_df.shape[0]))
print("\n")
print("Sample records: \n")
result_df.head(5)


<p style = 'font-size:16px;font-family:Arial'>
The table contains both the customer (Customer_id) and Path (the dominant path selected)</p>
    
<p style = 'font-size:16px;font-family:Arial'>This saved segment can be used as input for further analysis, such as clustering to see if there are any commonalities across the customers, products searched or potentially as input into a customer care process to reach out to the customers to acknowledge their bad experience with our online store.</p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>10. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>

In [ ]:
eng.execute('DROP TABLE CustomersOnPath;')

In [ ]:
eng.execute('DROP TABLE npath_data;')

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_DigitalEvents');" 
#Takes 3 seconds

In [ ]:
# Removing the connection and clearing session
remove_context()

<hr>
<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>Conclusion</b>

<p style = 'font-size:16px;font-family:Arial'>As you can see from this brief demonstration, the Vantage nPath Analytic function is an effective way to conduct path analysis, such as the one we just viewed.   Path analysis can span multiple topics and crosses industries.</p> 

<ul style = 'font-size:16px;font-family:Arial'><b>Examples include:</b>
    <li>Customer paths to complaints</li>
<li>Paths to part failure</li>
<li>Paths to Disease</li>
<li>Path to Churn</li>
</ul>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Links</b>
<li><a href = 'https://docs.teradata.com/reader/eteIDCTX4O4IMvazRMypxQ/uDjppX7PJInABCckgu~KFg'>Teradata Python Package User Guide </a> </li>
<li><a href = 'https://docs.teradata.com/reader/GsM0pYRZl5Plqjdf9ixmdA/MzdO1q_t80M47qY5lyImOA'>Teradataml Python Reference </a></li>
<li><a href = 'https://docs.teradata.com/reader/CWVY0AJy8wyyf7Sm0EsK~w/wjkE42ypEfeMkRFOIqVXfQ'>Teradata nPath Function Reference:</a></li>
<li><a href = 'https://docs.teradata.com/reader/CWVY0AJy8wyyf7Sm0EsK~w/RNbOiUg9~r~cxSZHrR~sFQ'>Teradata Sessionize Function Reference</a></li>
<li><a href = 'https://pandas.pydata.org/docs/user_guide/index.html'>Python Pandas Reference</a></li>
<li><a href = 'https://plotly.com/'>Plotly Reference</a></li>
</p>    

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">Copyright © Teradata Corporation - 2023. All Rights Reserved.</footer>